In [ ]:
import os, cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
from tqdm import tqdm

In [ ]:
"""
This notebook should only be run after:
1) Fiji pre-processing (sorting of images into 'actin' folder)
2) Segmentation using ResNet50
3) MATLAB analysis
"""

rootdir = '../Test_data' 
output_folder = 'output'  #  Folder will be created in same directory as notebook


#  Find folders that have been segmented
folders = [i for i,d,_ in os.walk(rootdir, topdown=True) 
           if os.path.basename(i).endswith('actin')
           if '14_18' in os.listdir(i)
           if 'live_data' not in i]

actin_folders = []
for folder in folders:
    csv_path = os.path.join(folder, '14_18', 'Results.csv')
    if os.path.isfile(csv_path):
        df = pd.read_csv(csv_path)
        if len(list(df.columns)) != 8:
            print(f"Warning: {csv_path} contains less data than expected.")
    actin_folders.append(folder)

In [ ]:
"""
Get only fixed samples. 
Omit folders that contain cell indices eg. "0001", "0002" etc. (typical of live samples)

"""
folders = []
for folder in actin_folders:
    base_folder = folder.strip('actin')[:-1]
    try:
        int(os.path.basename(base_folder))
    except:
        folders.append(base_folder)
folders = sorted(folders)    
treatments = [os.path.basename(i) for i in folders]

In [ ]:
for folder, treatment in zip(folders, treatments):
    print(treatment, folder)

In [ ]:
def process_ring(dist_from_edge, ring_width_micron=4, folder_list=folders, treatments=treatments):
    folder_string = f"{dist_from_edge}_{dist_from_edge + ring_width_micron}"
    df_list = []
    for folder, treatment in zip(folder_list, treatments):
        csvpath = os.path.join(folder, 'actin', folder_string, 'Results.csv')
        if not os.path.isfile(csvpath):
            print(f"CSV path error: {folder}")
        temp = pd.read_csv(csvpath, index_col=None)
        temp["Treatment"] = treatment
        df_list.append(temp)
    df = pd.concat(df_list)
    df.columns = ['File_index', 'Fiber_ID', 'Raw_angle', 'Edge_x', 'Edge_y', 'Cell_area', 'Cell_x', 'Cell_y', 'Treatment']
    df = df.loc[np.abs(df.Raw_angle) <= 68].copy()  #  Omit obvious transverse/ventral fibres
    df = df.dropna()
    
    if len(df) == 0:
        print(f'empty df: {folder_string}')
        return None
    
    #  Omit cells with overly large or small cell masks (> 2000 microns square, <1700 microns square)
    min_area = 1700 / np.square(pixel_size)
    max_area = 2100 / np.square(pixel_size)  #  2000 --> 2100
    df = df.loc[(df.Cell_area < max_area) & (df.Cell_area > min_area)]
    
    #  Cell-centroid to RF segment peripheral endpoint distance  (h_i)
    df['CentroidEdge_distance'] = np.sqrt(
        np.square(df.Cell_x - df.Edge_x) 
        + np.square(df.Cell_y - df.Edge_y))
    
    #  To correct for the inflation of cell area due to lamellopidia, reduce cell area by 63.353µm^2
    #  which is the average difference between measured and expected cell area. 
    df['Cell_radius'] = np.sqrt(
        (df.Cell_area - (63.353 / pixel_size ** 2)) / np.pi)  # in pixels
    df['Angle'] = np.rad2deg(np.arcsin(
        df.CentroidEdge_distance * np.sin(np.deg2rad(df.Raw_angle)) / df.Cell_radius))
    return df

In [ ]:
ring_outer_range = np.arange(0, 16, 2)  #  0, 2, 4, ..., 14
ring_width = 4
pixel_size = 0.138502  # µm/pixel
means = pd.DataFrame() 
counts = pd.DataFrame()
for ring_outer in tqdm(ring_outer_range):  
    df_ring = process_ring(dist_from_edge=ring_outer, ring_width_micron=ring_width)  #  Get data from ring
    group = df_ring.groupby(['Treatment', 'File_index'])
    means[f'{ring_outer}-{ring_outer + ring_width}µm'] = group.mean().Angle  # Add mean data to dataframe
    counts[f'{ring_outer}-{ring_outer + ring_width}µm'] = group.count().Angle

means = means.reset_index()
counts = counts.fillna(0).reset_index()

In [ ]:
#  Check output
print(means.head())
print("\nTreatments: ")
for treatment in means.Treatment.unique():
    print(treatment)

In [ ]:
ind_output_folder = os.path.join(output_folder, 'CSVs')
os.makedirs(ind_output_folder, exist_ok=True)
cols = [i for i in means.columns if i.endswith('µm')]

#  Save summary output
means.to_csv(os.path.join(output_folder, 'All_angles_data.csv'), index=False)
g = means.groupby('Treatment')
g.mean()[cols].reset_index().to_csv(os.path.join(output_folder, 'Means.csv'), index=False)
g.std()[cols].reset_index().to_csv(os.path.join(output_folder, 'Std.csv'), index=False)
g.sem()[cols].reset_index().to_csv(os.path.join(output_folder, 'Sem.csv'), index=False)

#  Save data in CSVs sorted by treatment
for treatment in treatments:
    save_path = os.path.join(ind_output_folder, f"{treatment}.csv")
    g.get_group(treatment).to_csv(save_path, index=False)

In [ ]:
count_output_folder = os.path.join(output_folder, 'Count')
count_ind_output_folder = os.path.join(count_output_folder, 'CSVs')
os.makedirs(count_ind_output_folder, exist_ok=True)

#  Save summary count output
counts.to_csv(os.path.join(count_output_folder, 'All_counts_data.csv'), index=False)
g_counts = counts.groupby('Treatment')
g_counts.mean()[cols].to_csv(os.path.join(count_output_folder, 'Count_means.csv'), index=False)
g_counts.std()[cols].to_csv(os.path.join(count_output_folder, 'Count_std.csv'), index=False)
g_counts.sem()[cols].to_csv(os.path.join(count_output_folder, 'Count_sem.csv'), index=False)

#  Save data in CSVs sorted by treatment
for treatment in treatments:
    save_path = os.path.join(count_ind_output_folder, f"{treatment}.csv")
    g_counts.get_group(treatment).to_csv(save_path, index=False)